In [36]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.exceptions import NotFittedError

In [76]:
recommend = pd.read_csv("Set 2 Prediction Results.csv")
recommend.head()

,Unnamed: 0,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Predicted Loan Default
0,0,f738779f-c726-40dc-92cf-689d73af533d,ded0b3c3-6bf4-4091-8726-47039f2c1b90,611314.0,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,0.0,9.0,0.0,621908.0,1058970.0,0.0,0.0,0
1,1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1630e6e3-34e3-461a-8fda-09297d3140c8,266662.0,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,0.0,11.0,0.0,679573.0,904442.0,0.0,0.0,0
2,2,f7744d01-894b-49c3-8777-fc6431a2cff1,2c60938b-ad2b-4702-804d-eeca43949c52,153494.0,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10.0,10.0,0.0,38532.0,388036.0,0.0,0.0,0
3,3,83721ffb-b99a-4a0f-aea5-ef472a138b41,12116614-2f3c-4d16-ad34-d92883718806,176242.0,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,27.0,16.0,1.0,156940.0,531322.0,1.0,0.0,0
4,4,08f3789f-5714-4b10-929d-e1527ab5e5a3,39888105-fd5f-4023-860a-30a3e6f5ccb7,321992.0,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,44.0,14.0,0.0,359765.0,468072.0,0.0,0.0,0


In [73]:
recommend['Years in current job'].unique()

array(['10+ years', '2 years', '1 year', '< 1 year', '5 years', '7 years',
       '3 years', '6 years', '8 years', '4 years', '9 years'],
      dtype=object)

In [94]:
# function for the recommendation 1 is no deafult, 0 default
def loan_recommendations(row):
    loan_amount = row['Current Loan Amount']
    credit_score = row['Credit Score']
    predicted_default = row['Predicted Loan Default']
    income = row["Annual Income"]

    if predicted_default == 1:
        return "Loan Approved"
    else:
        if  300 < credit_score < 579:
            recommendation =  'Loan Between 10,000 - 50,000'
            min_amount, max_amount = 10000, 50000

        elif 580 <= credit_score < 669:
            recommendation = "Loan Between 50,000 - 150,000"
            min_amount, max_amount = 50000, 150000
            
        elif 670 <= credit_score < 739:
            recommendation = "Loan upto 250,000"
            min_amount, max_amount = 0, 250000
            
        elif 740 <= credit_score < 799:
           recommendation = "Loan upto 500,000"
           min_amount, max_amount = 0, 500000
            
        elif credit_score >= 800:
            recommendation = "Loan upto 1,000,000"
            min_amount,max_amount = 0, 1000000
            
        else:
            return "Loan Rejected"
        
        # If annual income is less that 100,000 the loan will likely be rejected
        if income <= 100000:
            return "Loan Rejected, likely to default"
        
        # if the loan amount is more than max loan amount loan likely to be rejected
        if min_amount <= loan_amount <= max_amount:
            return "Loan Rejected, likely to default"
        
        # if years of experience are less than 1 year the loan will likely be rejected
        if row['Years in current job'] == "< 1 year":
            return "Loan rejected, likely to default"
        
        # if the debt to income ratio is more than 36% the loan will likely be rejected
        row["Monthly Income"] = income/12
        row["DTI Ratio"] = row["Monthly Debt"]/row["Monthly Income"]

        if row['DTI Ratio'] > 0.36:
            return "Loan rejected, due to high debt-income ratio"
        
        # if you are delinquent for more than 90 days (assumption column used is days not months)
        if row["Months since last delinquent"] >= 90:
            return "Loan rejected, likely to default"

        return recommendation

In [95]:
recommend['Recommendation'] = recommend.apply(loan_recommendations, axis=1)

recommend.sample(10)

,Unnamed: 0,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,...,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Predicted Loan Default,Recommendation
9585,9585,c837b1e9-4171-4488-b738-cfed44a3fab2,0e84a2c7-2696-4f29-a715-16d1971afd52,777326.0,Long Term,674.00000,2.090684e+06,10+ years,Home Mortgage,Debt Consolidation,...,32.9,59.0,16.0,0.0,739214.0,1050214.0,0.0,0.0,0,"Loan upto 250,000"
9292,9292,bd5e7e66-70c8-4540-a049-eb8a7e51d03a,f8763468-05fc-49c1-881a-4219603c3742,195888.0,Short Term,734.00000,3.305582e+06,< 1 year,Rent,Home Improvements,...,12.6,0.0,11.0,0.0,370139.0,603636.0,0.0,0.0,0,"Loan Rejected, likely to default"
3243,3243,6f89d07b-ae9e-4ba7-a60e-0c9ae6197de8,91014e5b-3f97-46c8-a122-86dfd56636f2,603548.0,Long Term,715.00000,1.116972e+06,3 years,Rent,Buy a Car,...,6.0,0.0,14.0,0.0,74480.0,342232.0,0.0,0.0,1,Loan Approved
3031,3031,e6d00770-c2b3-422f-9b57-37a357619034,55d26c44-f355-45ed-958d-a037f9d812b8,173910.0,Short Term,719.00000,4.198240e+05,7 years,Own Home,major_purchase,...,9.0,0.0,12.0,0.0,140372.0,460416.0,0.0,0.0,0,"Loan Rejected, likely to default"
556,556,658e7204-9640-457e-868e-d2afda78ceed,d5f11c7f-829a-4e46-92e5-2475dbbd8442,183040.0,Short Term,740.00000,4.835310e+05,6 years,Home Mortgage,Debt Consolidation,...,19.8,79.0,7.0,0.0,313842.0,506132.0,0.0,0.0,0,"Loan Rejected, likely to default"
3259,3259,5cbdbf85-9b5b-43af-a372-835d9e8f5cc2,1667abf7-7fd7-49fd-ac15-b48787cba72a,211860.0,Short Term,1077.99152,1.369106e+06,9 years,Rent,Debt Consolidation,...,11.0,81.0,4.0,0.0,45961.0,124366.0,0.0,0.0,1,Loan Approved
1726,1726,99878abc-439a-4ceb-80aa-4ed6e17e2fb1,bb61b766-2369-4af7-8c73-4f744bdd030d,266684.0,Short Term,740.00000,1.151628e+06,10+ years,Home Mortgage,Debt Consolidation,...,13.8,6.0,6.0,0.0,295811.0,417714.0,0.0,0.0,0,"Loan Rejected, likely to default"
7752,7752,128b920f-f19e-4366-8f05-8de7754b8acf,b3a21e94-0333-4b7e-8185-fa7d49c96f9e,218152.0,Short Term,7070.00000,1.168101e+06,10+ years,Home Mortgage,Debt Consolidation,...,17.6,17.0,11.0,0.0,228228.0,307648.0,0.0,0.0,1,Loan Approved
3795,3795,c6f244a6-23a4-4f42-9bf4-176946cc8a84,ffbc8947-175f-412f-be3c-b5dc51b2a3ca,263054.0,Short Term,739.00000,1.419870e+06,10+ years,Own Home,Debt Consolidation,...,16.0,27.0,9.0,0.0,185896.0,319814.0,0.0,0.0,0,Loan Rejected
1400,1400,17de5dad-a6c3-4ebc-a13f-84af2ba2ab51,4a079b4e-b8a7-47f7-914c-b8225de5787c,245036.0,Short Term,1077.99152,1.369106e+06,6 years,Home Mortgage,other,...,37.0,0.0,36.0,0.0,202027.0,4977170.0,0.0,0.0,0,"Loan Rejected, likely to default"


In [92]:
recommend.head(10)

,Unnamed: 0,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,...,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Predicted Loan Default,Recommendation
0,0,f738779f-c726-40dc-92cf-689d73af533d,ded0b3c3-6bf4-4091-8726-47039f2c1b90,611314.0,Short Term,747.00000,2.074116e+06,10+ years,Home Mortgage,Debt Consolidation,...,21.8,0.0,9.0,0.0,621908.0,1058970.0,0.0,0.0,0,"Loan upto 500,000"
1,1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1630e6e3-34e3-461a-8fda-09297d3140c8,266662.0,Short Term,734.00000,1.919190e+06,10+ years,Home Mortgage,Debt Consolidation,...,19.4,0.0,11.0,0.0,679573.0,904442.0,0.0,0.0,0,"Loan upto 250,000"
2,2,f7744d01-894b-49c3-8777-fc6431a2cff1,2c60938b-ad2b-4702-804d-eeca43949c52,153494.0,Short Term,709.00000,8.711120e+05,2 years,Rent,Debt Consolidation,...,12.5,10.0,10.0,0.0,38532.0,388036.0,0.0,0.0,0,"Loan Rejected, likely to default"
3,3,83721ffb-b99a-4a0f-aea5-ef472a138b41,12116614-2f3c-4d16-ad34-d92883718806,176242.0,Short Term,727.00000,7.800830e+05,10+ years,Rent,Debt Consolidation,...,16.5,27.0,16.0,1.0,156940.0,531322.0,1.0,0.0,0,"Loan Rejected, likely to default"
4,4,08f3789f-5714-4b10-929d-e1527ab5e5a3,39888105-fd5f-4023-860a-30a3e6f5ccb7,321992.0,Short Term,744.00000,1.761148e+06,10+ years,Home Mortgage,Debt Consolidation,...,26.0,44.0,14.0,0.0,359765.0,468072.0,0.0,0.0,0,"Loan Rejected, likely to default"
5,5,a4957169-d809-44cc-847b-975400bc8d11,6878d414-6a22-4712-ae43-9b3f798e463a,202928.0,Short Term,741.00000,7.603800e+05,1 year,Rent,Debt Consolidation,...,13.8,0.0,6.0,0.0,258647.0,476872.0,0.0,0.0,0,"Loan Rejected, likely to default"
6,6,43467302-94fe-494b-b52f-3fd891fea71c,48113a98-a4a0-4956-b57d-f0ce344826fb,621786.0,Long Term,733.00000,1.783606e+06,10+ years,Home Mortgage,Debt Consolidation,...,15.3,0.0,42.0,0.0,281599.0,1449162.0,0.0,0.0,0,"Loan upto 250,000"
7,7,930c7cb3-6086-434a-9547-3ed41c181552,19941661-98e2-4800-93c9-a0e92057c813,266794.0,Long Term,1077.99152,1.369106e+06,< 1 year,Own Home,Debt Consolidation,...,5.8,0.0,9.0,0.0,233206.0,342232.0,0.0,0.0,1,Loan Approved
8,8,0b2f1b66-741e-4e37-a929-99926cdc9e9a,6a1adeda-079b-49e5-ac7c-91828f2806a0,202466.0,Short Term,736.00000,1.068617e+06,5 years,Rent,Debt Consolidation,...,20.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1,Loan Approved
9,9,d08f3a5e-93df-40e7-bdd8-cba59180bddf,4080a828-a61a-4f04-a627-397f4319500c,266288.0,Long Term,683.00000,2.031518e+06,2 years,Rent,Debt Consolidation,...,24.4,56.0,8.0,2.0,31445.0,251130.0,2.0,0.0,0,"Loan upto 250,000"


In [97]:
recommend.to_csv("Loan Recommendations.csv")